# Laboratorium 2 - Convolutional Neural Network cz. II

# *Zadanie 1* - Convolutional Neural Network (Max 1h 15min):

**Zaimplementuj** model Convolutional Neural Networks (CNNs) do klasyfikacji:

* Reuters
* IMDB
* Boston Housing price

Wszystkie wspomniane zbiory danych znajduja się module Datasets Keras.

Dla datasetu Reuters może przydac sie poniższa implemtacja. Zapoznaj się z nia i dowiedz czym sa warstwy:
* Conv1D,
* Embedding,
* GlobalMaxPooling1D.

Czym jest **Tokenizer**? 

```
# Load the Reuters dataset
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=10000)

# Convert the data to one-hot encoding
tokenizer = Tokenizer(num_words=10000)
X_train = tokenizer.sequences_to_matrix(X_train, mode='binary')
X_test = tokenizer.sequences_to_matrix(X_test, mode='binary')
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Define the model architecture
model = Sequential()
model.add(Embedding(10000, 128, input_length=10000))
model.add(Conv1D(64, 3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(46, activation='softmax'))
```

Natomiast w przypadku IMDB przydać się możę coś podobnego:

```
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the IMDB dataset and keep only the top 5000 most frequent words
word_index = imdb.get_word_index()
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

# Pad sequences to the same length
max_review_length = 500
X_train = pad_sequences(X_train, maxlen=max_review_length)
X_test = pad_sequences(X_test, maxlen=max_review_length)

```

Czym jest **pad_sequences** ?

# Autoenkodery

**Autokodery**, zwane również autoenkoderami, pobierają dane wejściowe, przekształcają je w pewną reprezentację wewnętrzną, a następnie odtwarzają wynik przypominający dane wejściowe. Dokładniej, autoenkodery odnoszą się do sieci neuronowych, które potrafią uczyć się reprezentacji danych wejściowych (kodowania).

Autoenkodery zawsze składają się z dwóch elementów:

* kodera, który przekształca dane wejściowe do postaci reprezentacji,
* dekodera, który przekształca dane z postaci reprezentacji na dane wyjściowe.

Autoenkodery to rodzaj sztucznych sieci neuronowych, które są zdolne do nauki reprezentacji danych wejściowych. Najczęściej wyróżnia się dwa rodzaje autoenkoderów - niedopełniony i przepełniony. **Niedopełniony autoenkoder** ma mniej warstw ukrytych niż liczba danych wejściowych. To wymusza na nim znalezienie innej reprezentacji danych, ponieważ nie może po prostu skopiować i przekazać dalej danych wejściowych. **Przepełniony autoenkoder** ma więcej warstw ukrytych niż liczba danych wejściowych. Dzięki temu może uzyskać więcej cech. Jednak, w takim przypadku autoenkoder może się nauczyć po prostu przekazywać dane do kolejnych węzłów ukrytych, pomijając wiele innych, co uniemożliwia dodatkową ekstrakcję cech.





Podział autoenkoderów:

* Autoenkodery stosowe lub głębokie

Autoenkodery, podobnie jak sieci neuronowe, mogą składać się z wielu warstw ukrytych. Dodawanie kolejnych warstw ma na celu wyuczenie modelu bardziej skomplikowanych zależności i kodowań. W przypadku głębokich autoenkoderów najczęściej stosuje się symetryczną architekturę, zwana "kanapką", zmniejszając kolejną liczbę warstw ukrytych o połowę.

* Autoenkodery odszumiające (ang. denoising autoencoders)

Inną ciekawą metodą do wydobywania cech i uniknięcia nadmiernego dopasowania autoenkodera do danych jest dodanie szumu lub zamaskowanie niektórych wartości wejściowych w sposób stochastyczny. Następnie model jest trenowany do odzyskiwania oryginalnych danych wejściowych (uwaga: tych nieuszkodzonych bez dodanego szumu). Pierwsze koncepcje wykorzystywania autoenkoderów do odszumiania pochodzą z lat 80. Bardziej znana jest jednak praca naukowa Pascala Vincenta z Uniwersytetu w Montrealu w Kanadzie, który w 2008 roku pokazał, że autoenkodery są świetnym narzędziem do wydobywania cech.

* Autoenkodery rzadkie (ang. sparse autoencoders)

Rzadkie autoenkodery nakładają ograniczenie na funkcję aktywacji warstw ukrytych, aby uniknąć nadmiernego dopasowania i poprawić odporność. Ten rodzaj autoenkodera pozwala na aktywację tylko niewielkiej liczby jednostek w ukrytej warstwie w tym samym czasie. Innymi słowy, jeden ukryty neuron powinien być nieaktywny przez większość czasu.

* Autoenkodery kurczliwe (ang. contractive autoencoders)

Podobnie jak rzadkie autoenkodery, modyfikują funkcję straty, ale w ten sposób, aby ukarać reprezentację, która jest zbyt wrażliwa na dane wejściowe. Tym samym poprawia odporność na małe zaburzenia wokół punktów danych treningowych. Innymi słowy, dwa podobne do siebie przykłady wejściowe muszą mieć podobne kodowanie.

* Autoenkodery wariacyjne (ang. variational autoencoders)

VAE należą do klasy autoenkoderów generatywnych, czyli takich, które umieją tworzyć nowe dane przypominające te w zbiorze uczącym. Dodatkowo VAE są autoenkoderami probabilistycznymi, czyli generują częściowo losowe wyniki nawet po wyuczeniu modelu. Powiedziałbym, że mają standardową architekturę autoenkoderów: po koderze składającym się często z kilku warstw ukrytych mamy dekoder. Natomiast mamy tutaj drobną zmianę: koder nie generuje bezpośrednio nas

# Przykładowe implementacje Autoenkoderów

## Autoenkodery stosowe lub głęboki

```
from keras.layers import Input, Dense
from keras.models import Model
import numpy as np

# Wymiary obrazków MNIST
input_dim = 784

# Wyznaczamy wymiary warstw kodera i dekodera
encoding_dim1 = 256
encoding_dim2 = 128
encoding_dim3 = 64

# Definiujemy warstwy kodera
input_img = Input(shape=(input_dim,))
encoded = Dense(encoding_dim1, activation='relu')(input_img)
encoded = Dense(encoding_dim2, activation='relu')(encoded)
encoded = Dense(encoding_dim3, activation='relu')(encoded)

# Definiujemy warstwy dekodera
decoded = Dense(encoding_dim2, activation='relu')(encoded)
decoded = Dense(encoding_dim1, activation='relu')(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

# Tworzymy model autoenkodera
autoencoder = Model(input_img, decoded)

# Kompilujemy model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Wczytujemy zbiór danych MNIST
from keras.datasets import mnist
(x_train, _), (x_test, _) = mnist.load_data()

# Normalizujemy dane i przekształcamy na wektor
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Trenujemy model
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

# Wizualizacja wyników
import matplotlib.pyplot as plt

# Zastosuj model na zestawie testowym
decoded_imgs = autoencoder.predict(x_test)

# Wypisz kilka obrazków testowych i odpowiadające im dekodowane wersje
n = 10  # Ile obrazków chcesz wyświetlić
plt.figure(figsize=(20, 4))
for i in range(n):
    # Obrazek testowy
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Dekodowana wersja obrazka
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()
```

## Autoenkodery odszumiające (ang. denoising autoencoders)

```
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
import numpy as np

# Wymiary obrazków MNIST
input_dim = 784

# Wyznaczamy wymiary warstw kodera i dekodera
encoding_dim1 = 256
encoding_dim2 = 128
encoding_dim3 = 64

# Definiujemy warstwy kodera
input_img = Input(shape=(input_dim,))
encoded = Dense(encoding_dim1, activation='relu')(input_img)
encoded = Dense(encoding_dim2, activation='relu')(encoded)
encoded = Dense(encoding_dim3, activation='relu')(encoded)

# Definiujemy warstwy dekodera
decoded = Dense(encoding_dim2, activation='relu')(encoded)
decoded = Dense(encoding_dim1, activation='relu')(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

# Tworzymy model autoenkodera
autoencoder = Model(input_img, decoded)

# Kompilujemy model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Wczytujemy zbiór danych MNIST
(x_train, _), (x_test, _) = mnist.load_data()

# Normalizujemy i przekształcamy dane
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Dodajemy szum gaussowski do obrazków
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

# Przekształcamy dane na wektory
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
x_train_noisy = x_train_noisy.reshape((len(x_train_noisy), np.prod(x_train_noisy.shape[1:])))
x_test_noisy = x_test_noisy.reshape((len(x_test_noisy), np.prod(x_test_noisy.shape[1:])))

# Trenujemy model na danych z szumem
autoencoder.fit(x_train_noisy, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test_noisy, x_test))

# Wizualizacja wyników
import matplotlib.pyplot as plt

# Zastosuj model na zestawie testowym
decoded_imgs = autoencoder.predict(x_test_noisy)

# Wypisz kilka obrazków testowych i odpowiadające im dekodowane wersje
n = 10  # Ile obrazków chcesz wyświetlić
plt.figure(figsize=(20, 4))
for i in range(n):
    # Obrazek testowy
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Odpowiadający mu obrazek zdekodowany przez model
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
plt.show()
```

## Autoenkodery rzadkie (ang. sparse autoencoders)

```
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras import regularizers
import numpy as np

# Wymiary obrazków MNIST
input_dim = 784

# Wyznaczamy wymiary warstw kodera i dekodera
encoding_dim1 = 256
encoding_dim2 = 128
encoding_dim3 = 64

# Parametr regularyzacji rzadkości
sparsity_factor = 0.1

# Definiujemy warstwy kodera
input_img = Input(shape=(input_dim,))
encoded = Dense(encoding_dim1, activation='relu', activity_regularizer=regularizers.l1(sparsity_factor))(input_img)
encoded = Dense(encoding_dim2, activation='relu', activity_regularizer=regularizers.l1(sparsity_factor))(encoded)
encoded = Dense(encoding_dim3, activation='relu', activity_regularizer=regularizers.l1(sparsity_factor))(encoded)

# Definiujemy warstwy dekodera
decoded = Dense(encoding_dim2, activation='relu')(encoded)
decoded = Dense(encoding_dim1, activation='relu')(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

# Tworzymy model autoenkodera
autoencoder = Model(input_img, decoded)

# Kompilujemy model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Wczytujemy zbiór danych MNIST
(x_train, _), (x_test, _) = mnist.load_data()

# Normalizujemy i przekształcamy dane
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Trenujemy model
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

# Wizualizacja wyników
import matplotlib.pyplot as plt

# Zastosuj model na zestawie testowym
decoded_imgs = autoencoder.predict(x_test)

# Wypisz kilka obrazków testowych i odpowiadające im dekodowane wersje
n = 10  # Ile obrazków chcesz wyświetlić
plt.figure(figsize=(20, 4))
for i in range(n):
    # Obrazek testowy
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Odpowiadający mu obrazek zdekodowany przez model
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
plt.show()
```

## Autoenkodery kurczliwe (ang. contractive autoencoders)



```
from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers
import numpy as np

# Load MNIST dataset
from keras.datasets import mnist
(x_train, _), (x_test, _) = mnist.load_data()

# Normalize and flatten input data
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Define contractive autoencoder model
input_img = Input(shape=(784,))
encoded = Dense(128, activation='relu', activity_regularizer=regularizers.l1(10e-5))(input_img)
decoded = Dense(784, activation='sigmoid')(encoded)

# Create model
autoencoder = Model(input_img, decoded)

# Compile model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train model
autoencoder.fit(x_train, x_train,
                epochs=10,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

# Encode and decode test set
encoded_imgs = autoencoder.predict(x_test)
```

## Autoenkodery wariacyjne (ang. variational autoencoders)


```
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K
import numpy as np

# Define the dimensions of the latent space
latent_dim = 2

# Define the input shape
input_shape = (784,)

# Define the encoder architecture
input_layer = Input(shape=input_shape)
hidden_layer = Dense(512, activation='relu')(input_layer)
z_mean = Dense(latent_dim)(hidden_layer)
z_log_var = Dense(latent_dim)(hidden_layer)

# Define the sampling function
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=1.)
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

# Use the sampling function to sample from the latent space
z = Lambda(sampling)([z_mean, z_log_var])

# Define the decoder architecture
decoder_input = Input(shape=(latent_dim,))
decoder_hidden = Dense(512, activation='relu')(decoder_input)
decoder_output = Dense(784, activation='sigmoid')(decoder_hidden)

# Define the models
encoder = Model(input_layer, [z_mean, z_log_var, z], name='encoder')
decoder = Model(decoder_input, decoder_output, name='decoder')
vae_output = decoder(encoder(input_layer)[2])
vae = Model(input_layer, vae_output, name='vae')

# Define the loss function
reconstruction_loss = binary_crossentropy(input_layer, vae_output)
kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
vae_loss = K.mean(reconstruction_loss + kl_loss)

# Compile the model
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess the data
x_train = x_train.reshape(x_train.shape[0], np.prod(x_train.shape[1:]))
x_train = x_train.astype('float32') / 255.
x_test = x_test.reshape(x_test.shape[0], np.prod(x_test.shape[1:]))
x_test = x_test.astype('float32') / 255.

# Train the model
vae.fit(x_train, epochs=2, batch_size=128, validation_data=(x_test, None))


import matplotlib.pyplot as plt

def display_vae_results(vae, x_test):
    # Encode the test images
    encoder = vae.get_layer('encoder')
    z_mean, _, _ = encoder.predict(x_test)

    # Generate new images by sampling from the learned distribution
    decoder = vae.get_layer('decoder')
    n = 15  # number of images to generate
    digit_size = 28
    figure = np.zeros((digit_size * n, digit_size * n))

    # Sample from the learned distribution
    grid_x = np.linspace(-4, 4, n)
    grid_y = np.linspace(-4, 4, n)[::-1]
    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi]])
            x_decoded = decoder.predict(z_sample)
            digit = x_decoded[0].reshape(digit_size, digit_size)
            figure[i * digit_size: (i + 1) * digit_size,
                   j * digit_size: (j + 1) * digit_size] = digit

    # Plot the generated images
    plt.figure(figsize=(10, 10))
    plt.imshow(figure, cmap='Greys_r')
    plt.show()

# Generate and display new images
display_vae_results(vae, x_test)

```

# *Zadanie 2* - Podstawy Autoenkoderów (Max 1h 15min):

**Zaimplementuj** wszysytkie typy autoenkoderów dla zbioru danych:

* CIFAR-10 (wykorzystaj wbudowany modul z biblioteki Keras)

Wykorzystaj inne Optymalizatory oraz bardziej zaawansowana architekture niż w przykładach. Jak należy interpretować osiagniete wyniki dla CIFAR-10? W celu zrozumienia autoencoderów zapoznaj się z załaczonymi materiałami.

# Materiały

Autoenkodery:
* https://www.unite.ai/what-is-an-autoencoder/ (wraz z rodzajami) 
* https://www.tensorflow.org/tutorials/generative/autoencoder
* https://www.mygreatlearning.com/blog/autoencoder/ 
* https://towardsdatascience.com/auto-encoder-what-is-it-and-what-is-it-used-for-part-1-3e5c6f017726 
* https://iq.opengenus.org/types-of-autoencoder/
* https://www.jeremyjordan.me/autoencoders/ 
* https://www.deeplearningbook.org/contents/autoencoders.html
* https://www.mygreatlearning.com/blog/autoencoder/
* https://www.tensorflow.org/tutorials/generative/autoencoder?hl=pl
* https://medium.com/pytorch/implementing-an-autoencoder-in-pytorch-19baa22647d1
* https://www.tutorialspoint.com/how-to-implementing-an-autoencoder-in-pytorch
* https://github.com/AlexPasqua/Autoencoders 

Youtube:

* https://www.youtube.com/watch?v=JoR5HCs0n0s
* https://www.youtube.com/watch?v=1h-KUgGSrsk 
* https://www.youtube.com/watch?v=8wrLjnQ7EWQ